In [1]:
%load_ext memory_profiler

In [2]:
%memit

peak memory: 39.58 MiB, increment: 0.07 MiB


In [4]:
%load_ext autoreload
%autoreload 2
 
%matplotlib inline
# Set below to 0 or 1 for GPU
%env CUDA_VISIBLE_DEVICES=1
from __future__ import absolute_import, division, print_function
import json
import os
if os.path.isfile("config.json"):
    with open("config.json", "r") as f:
        config = json.load(f)
else:
    print("Please run setup.py in this directory before running any .ipynb's.")

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import tensorflow as tf
import time
import shutil
import pickle
from collections import OrderedDict, defaultdict

from utils.bags import *
from utils.learning import *
from multimodal_generative_model import *
from st_graph import *

config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: CUDA_VISIBLE_DEVICES=1


{u'catkin_ws_dir': u'/Users/borisi/catkin_ws',
 u'data_dir': u'data',
 u'feature_extraction_dict': u'slim',
 u'julia_pkg_dir': u'/Users/borisi/.julia/v0.6',
 u'models_dir': u'models'}

In [5]:
cvd = %env CUDA_VISIBLE_DEVICES
model_dir = os.path.join(config["models_dir"], ("GPU" + cvd if cvd else "CPU") + "_checkpoints")
# shutil.rmtree(model_dir, ignore_errors=True)

sc = tf.ConfigProto(device_count={'GPU': 1}, allow_soft_placement=True, log_device_placement=False)
rc = tf.estimator.RunConfig().replace(session_config=sc, model_dir=model_dir,
                                      save_summary_steps=100, save_checkpoints_steps=5000,
                                      keep_checkpoint_max=5000, tf_random_seed=None)
rc.environment = None    # required due to a bug in tf.contrib.learn.Experiment.train_and_evaluate
model_dir

u'models/GPU1_checkpoints'

In [6]:
# Edit model hyperparameters if desired, e.g.:
# hps.N = 2
# hps.K = 5
hps.values()

{'GMM_components': 16,
 'K': 5,
 'MLP_dropout_keep_prob': 0.9,
 'MVG_latent_dim': 32,
 'N': 2,
 'alpha': 1,
 'batch_size': 16,
 'dec_GMM_proj_MLP_dims': None,
 'dec_rnn_dim': [128],
 'dec_sample_model_prob_crossover': 20000,
 'dec_sample_model_prob_divisor': 6,
 'dec_sample_model_prob_final': 0.0,
 'dec_sample_model_prob_start': 0.0,
 'enc_rnn_dim_edge': [8],
 'enc_rnn_dim_edge_influence': [8],
 'enc_rnn_dim_future': [32],
 'enc_rnn_dim_history': [32],
 'enc_rnn_dim_multiple_inputs': [8],
 'fuzz_factor': 0.05,
 'grad_clip': 1.0,
 'k': 3,
 'k_eval': 50,
 'kl_crossover': 8000,
 'kl_decay_rate': 0.99995,
 'kl_exact': True,
 'kl_min': 0.07,
 'kl_sigmoid_divisor': 6,
 'kl_weight': 1.0,
 'kl_weight_start': 0.0001,
 'latent_type': 'discrete',
 'learning_decay_rate': 0.9999,
 'learning_rate': 0.001,
 'log_p_yt_xz_max': 50,
 'log_sigma_max': 10,
 'log_sigma_min': -10,
 'min_learning_rate': 1e-05,
 'minimum_history_length': 5,
 'optimizer': tensorflow.python.training.adam.AdamOptimizer,
 'optimi

In [7]:
%memit

peak memory: 193.60 MiB, increment: 0.00 MiB


# Data Loading

In [8]:
extract_dict = feature_extraction_dicts[config["feature_extraction_dict"]]
array_name = "trajectories_" + config["feature_extraction_dict"]
base_filename = os.path.join(config["data_dir"], array_name)

all_bags = pd.read_pickle(base_filename + ".pkl")
train_bags, eval_bags, test_bags = split_bag_df(all_bags, train_eval_test_split = [.8, .2, 0.], seed=123)

pos_dict = pos_dict_from_dataframe(train_bags, cols={'car1': ('x1', 'y1'), 'car2': ('x2', 'y2')}, 
                                   node_names_and_types={'car1': 'robot', 'car2': 'human'})

robot_node = STGNode('car1', 'robot')
STG = SpatioTemporalGraphCVAE(pos_dict=pos_dict, robot_node=robot_node)

train_data_dict = prepare_data(train_bags, extract_dict=extract_dict)
save_as_npz_and_hdf5(base_filename + "_train", train_data_dict["input_dict"])
human_node = STGNode('car2', 'human')
train_data_dict["labels"] = {convert_to_label_node(human_node): train_data_dict["labels"]}
train_data_dict["input_dict"][human_node] = train_data_dict["input_dict"]["car2"]
train_data_dict["input_dict"][robot_node] = train_data_dict["input_dict"]["car1"]

hps.add_hparam("nodes_standardization", {robot_node: {"mean": train_data_dict["cars_mean"],
                                        "std": train_data_dict["cars_std"]},
                                         human_node: {"mean": train_data_dict["human_mean"],
                                        "std": train_data_dict["human_std"]}})
hps.add_hparam("extras_standardization", {"mean": train_data_dict["extras_mean"],
                                          "std": train_data_dict["extras_std"]})
hps.add_hparam("labels_standardization", {convert_to_label_node(human_node): {"mean": train_data_dict["labels_mean"],
                                          "std": train_data_dict["labels_std"]}})

hps.add_hparam("pred_indices", train_data_dict["pred_indices"])
eval_data_dict = prepare_data(eval_bags, extract_dict=extract_dict)
save_as_npz_and_hdf5(base_filename + "_eval", eval_data_dict["input_dict"])

eval_data_dict["labels"] = {convert_to_label_node(human_node): eval_data_dict["labels"]}
eval_data_dict["input_dict"][human_node] = eval_data_dict["input_dict"]["car2"]
eval_data_dict["input_dict"][robot_node] = eval_data_dict["input_dict"]["car1"]

train_input_function = tf.estimator.inputs.numpy_input_fn(train_data_dict["input_dict"],
                                                          y = train_data_dict["labels"],
                                                          batch_size = hps.batch_size,
                                                          num_epochs = None,
                                                          shuffle = True)

eval_input_function = tf.estimator.inputs.numpy_input_fn(eval_data_dict["input_dict"],
                                                         y = eval_data_dict["labels"],
                                                         batch_size = 4,
                                                         num_epochs = 1,
                                                         shuffle = False)

In [9]:
hps.values()

{'GMM_components': 16,
 'K': 5,
 'MLP_dropout_keep_prob': 0.9,
 'MVG_latent_dim': 32,
 'N': 2,
 'alpha': 1,
 'batch_size': 16,
 'dec_GMM_proj_MLP_dims': None,
 'dec_rnn_dim': [128],
 'dec_sample_model_prob_crossover': 20000,
 'dec_sample_model_prob_divisor': 6,
 'dec_sample_model_prob_final': 0.0,
 'dec_sample_model_prob_start': 0.0,
 'enc_rnn_dim_edge': [8],
 'enc_rnn_dim_edge_influence': [8],
 'enc_rnn_dim_future': [32],
 'enc_rnn_dim_history': [32],
 'enc_rnn_dim_multiple_inputs': [8],
 'extras_standardization': {'mean': array([], dtype=float32),
  'std': array([], dtype=float32)},
 'fuzz_factor': 0.05,
 'grad_clip': 1.0,
 'k': 3,
 'k_eval': 50,
 'kl_crossover': 8000,
 'kl_decay_rate': 0.99995,
 'kl_exact': True,
 'kl_min': 0.07,
 'kl_sigmoid_divisor': 6,
 'kl_weight': 1.0,
 'kl_weight_start': 0.0001,
 'labels_standardization': {human/car2_label: {'mean': array([ 0.5703405 , -0.09813011], dtype=float32),
   'std': array([2.162645 , 1.2083925], dtype=float32)}},
 'latent_type': 'disc

In [10]:
laneswap_model = STG
nn = tf.estimator.Estimator(laneswap_model.model_fn, params=hps, config=rc, model_dir=model_dir)
experiment = tf.contrib.learn.Experiment(nn, train_input_function, eval_input_function,
                                         train_steps=10000, min_eval_frequency=1, eval_steps=None)

INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_session_config': device_count {
  key: "GPU"
  value: 1
}
allow_soft_placement: true
, '_keep_checkpoint_max': 5000, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x122852090>, '_save_checkpoints_steps': 5000, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', 'environment': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': u'models/GPU1_checkpoints', '_save_summary_steps': 100}


In [11]:
%memit

peak memory: 250.71 MiB, increment: 0.00 MiB


# Training

In [12]:
experiment.train_and_evaluate()

Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into models/GPU1_checkpoints/model.ckpt.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

INFO:tensorflow:Starting evaluation at 2018-03-28-04:20:05
INFO:tensorflow:Restoring parameters from models/GPU1_checkpoints/model.ckpt-1


KeyboardInterrupt: 

In [13]:
%memit

peak memory: 770.16 MiB, increment: 0.00 MiB


# Number of Parameters

In [14]:
experiment.estimator.get_variable_names()

['global_step',
 'human-robot/edge_encoder/rnn/multi_rnn_cell/cell_0/layer_norm_basic_lstm_cell/bias',
 'human-robot/edge_encoder/rnn/multi_rnn_cell/cell_0/layer_norm_basic_lstm_cell/kernel',
 'human/decoder/p_y_xz/initial_state_projection/cell_0/initial_c/bias',
 'human/decoder/p_y_xz/initial_state_projection/cell_0/initial_c/kernel',
 'human/decoder/p_y_xz/initial_state_projection/cell_0/initial_h/bias',
 'human/decoder/p_y_xz/initial_state_projection/cell_0/initial_h/kernel',
 'human/decoder/p_y_xz/rnn/corr_projection/bias',
 'human/decoder/p_y_xz/rnn/corr_projection/kernel',
 'human/decoder/p_y_xz/rnn/log_pi_projection/bias',
 'human/decoder/p_y_xz/rnn/log_pi_projection/kernel',
 'human/decoder/p_y_xz/rnn/log_sigma_projection/bias',
 'human/decoder/p_y_xz/rnn/log_sigma_projection/kernel',
 'human/decoder/p_y_xz/rnn/mu_projection/bias',
 'human/decoder/p_y_xz/rnn/mu_projection/kernel',
 'human/decoder/p_y_xz/rnn/multi_rnn_cell/cell_0/layer_norm_basic_lstm_cell/bias',
 'human/decoder

In [15]:
def print_num_params(experiment, level=2):
    variable_names = experiment.estimator.get_variable_names()
    if level == 0:
        # Total number of parameters
        num_params = np.sum([np.prod(experiment.estimator.get_variable_value(var_name).shape) for var_name in variable_names]).astype(int)
        print("Total number of parameters: {:,}".format(num_params))
    
    else:
        node_type_params = defaultdict(int)
        for variable_name in variable_names:
            key = '/'.join(variable_name.split('/')[:level])
            node_type_params[key] += np.prod(experiment.estimator.get_variable_value(variable_name).shape).astype(int)

        for (key, value) in node_type_params.iteritems():
            print("{}: {:,}".format(key, value))

    print("-"*40)
    
print_num_params(experiment, level=0)
print_num_params(experiment, level=1)
print_num_params(experiment, level=2)

Total number of parameters: 782,911
----------------------------------------
robot_robot: 5,440
optimizer: 521,930
global_step: 1
human: 254,868
human-robot: 672
----------------------------------------
robot_robot/future_encoder: 5,440
human/label_standardization: 4
global_step: 1
human/future_encoder: 4,928
human/encoder: 6,468
optimizer/robot_robot: 10,880
human/features_standardization: 12
human/decoder: 237,920
human-robot/edge_encoder: 672
optimizer/beta2_power: 1
human/total_edge_influence_encoder: 544
optimizer/human-robot: 1,344
optimizer/human: 509,704
human/history_encoder: 4,992
optimizer/beta1_power: 1
----------------------------------------


# Saving the Model

In [16]:
state_dim = train_data_dict["input_dict"][robot_node].shape[2]
extras_dim = train_data_dict["input_dict"]["extras"].shape[2]
ph = hps.prediction_horizon

with tf.Graph().as_default():
    serving_input_receiver_fn = tf.estimator.export.build_raw_serving_input_receiver_fn(
        {"robot/car1": tf.placeholder(tf.float32, shape=[1, None, state_dim], name="robot/car1"),
         "human/car2": tf.placeholder(tf.float32, shape=[1, None, state_dim], name="human/car2"),
         "extras": tf.placeholder(tf.float32, shape=[1, None, extras_dim], name="extras"),
         "traj_lengths": tf.placeholder(tf.int32, shape=[1], name="traj_lengths"),
         "robot/car1_future_x": tf.placeholder(tf.float32, shape=[None, ph, state_dim/2], name="robot/car1_future_x"),
         "robot/car1_future_y": tf.placeholder(tf.float32, shape=[None, ph, state_dim/2], name="robot/car1_future_y"),
         "sample_ct": tf.placeholder(tf.int32, shape=[1], name="sample_ct")})
    
    nn.export_savedmodel(config["models_dir"], serving_input_receiver_fn)

INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default', 'predictions']
INFO:tensorflow:Restoring parameters from models/GPU1_checkpoints/model.ckpt-1
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: models/temp-1522211010/saved_model.pb
